#########################################################<br>
 AIR CAVITY<br>
 IMPEDANCE PAROI<br>
 THIN FLEXIBLE STRUCTURE<br>
 XFEM<br>
 CB REDUCTION FOR AIR<br>
 MODAL REDUCTION FOR STRUCTURE<br>
#########################################################

good results / good management of reduced dofs on the air-porous interface

python -m cProfile [-o output_file] [-s sort_order] myscript.py

#########################################################<br>
Libraries<br>
#########################################################

In [138]:
import string
import time
import scipy
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
import pickle
import sys
sys.path.append('../librairies')
import silex_lib_xfem_acou_tet4
import silex_lib_gmsh
import silex_lib_dkt_fortran as silex_lib_dkt
import silex_lib_porous_tet4_fortran

In [139]:
import mumps
from mpi4py import MPI
comm = MPI.COMM_WORLD
nproc=comm.Get_size()
rank = comm.Get_rank()

In [140]:
class comm_mumps_one_proc:
    rank = 0
    def py2f(self):
        return 0
mycomm=comm_mumps_one_proc()

#########################################################<br>
To run it in parallel for several frequencies:<br>
export OPENBLAS_NUM_THREADS=1<br>
mpirun -np 20 python3 Main_xfem_fluid_flex_struc_CB_reduction_impedance_paroi_12.py<br>
<br>
To run it in sequentiel frequency per frequency with openblas in parrallel:<br>
export OPENBLAS_NUM_THREADS=10<br>
python3 Main_toto.py<br>
<br>
#########################################################

In [141]:
if rank==0:
    print ("time at the beginning of the computation:",time.ctime())

time at the beginning of the computation: Fri Feb 11 14:58:28 2022


############################################################<br>
############################################################<br>
             S T A R T   M A I N   P R O B L E M<br>
############################################################<br>
############################################################

############################################################<br>
Datas<br>
############################################################

In [142]:
mesh_file='geom/cavity12_with_porous_air'
results_file='results/cavity12_with_impedance_air_flexible_structure_CB_reduction_test'

fichier : results_file='results/cavity12_with_impedance_air_flexible_structure_CB_reduction'<br>
nb_mode_F = 400<br>
nb_mode_S = 70<br>
freq_ini     = 10.0<br>
freq_end     = 200.0<br>
nb_freq_step_per_proc=800

In [143]:
flag_write_gmsh_results=1

In [144]:
nb_mode_F = 200
nb_mode_S = 50
freq_ini     = 10.0
freq_end     = 200.0
nb_freq_step_per_proc=800

In [145]:
nb_freq_step = nb_freq_step_per_proc*nproc
deltafreq=(freq_end-freq_ini)/(nb_freq_step-1)

air

In [146]:
celerity=343.0 # ok
rho=1.21 # ok

shell structure

In [147]:
material_Struc=[]
material_Struc.append(75000.0e6) # E Young
material_Struc.append(0.33) # nu
material_Struc.append(5.0e-3) # thickness
material_Struc.append(2700.0) # rho

structure damping

In [148]:
modal_damping_S=0.0

impedance paroi : article Walid-JFD : CMAME 2008

In [149]:
d_imp_paroi = 50.0 # Pa.s/m
k_imp_paroi = 5.0e6 # Pa/m

############################################################<br>
Load fluid mesh<br>
############################################################

In [150]:
tic = time.perf_counter()

In [151]:
fluid_nodes    = silex_lib_gmsh.ReadGmshNodes(mesh_file+'.msh',3)
fluid_elements1,IdNodes1 = silex_lib_gmsh.ReadGmshElements(mesh_file+'.msh',4,1) # air, cavity + controlled volume
fluid_elements5,IdNodes5 = silex_lib_gmsh.ReadGmshElements(mesh_file+'.msh',4,5) # air, ONLY controlled volume
fluid_elements_S3,IdNodesS3 = silex_lib_gmsh.ReadGmshElements(mesh_file+'.msh',2,3)# air-porous interface surface
#fluid_elements2,IdNodes2 = silex_lib_gmsh.ReadGmshElements(mesh_file+'.msh',4,2) # porous, volume
#fluid_elements_S4,IdNodesS4 = silex_lib_gmsh.ReadGmshElements(mesh_file+'.msh',2,4) # porous, external surface

or i in range(len(IdNodesS3)):<br>
   print(fluid_nodes[IdNodesS3[i]-1,:])

In [152]:
fluid_nnodes   = fluid_nodes.shape[0]
fluid_nelem1   = fluid_elements1.shape[0]
#fluid_nelem2   = fluid_elements2.shape[0]
fluid_nelem3   = fluid_elements_S3.shape[0]
#fluid_nelem4   = fluid_elements_S4.shape[0]
fluid_nelem5   = fluid_elements5.shape[0]

In [153]:
fluid_nnodes1 = IdNodes1.shape[0]
#fluid_nnodes2 = IdNodes2.shape[0]
fluid_nnodes3 = IdNodesS3.shape[0]
#fluid_nnodes4 = IdNodesS4.shape[0]
fluid_nnodes5 = IdNodes5.shape[0]

In [154]:
fluid_ndof1     = fluid_nnodes1

In [155]:
if rank==0:
    print ("Number of nodes:",fluid_nnodes)
    print ("Number of elements in air:",fluid_nelem1)
    print ("Number of nodes in air:",fluid_nnodes1)
    print ("Number of nodes at interface:",fluid_nnodes3)

Number of nodes: 23382
Number of elements in air: 118211
Number of nodes in air: 21699
Number of nodes at interface: 407


renumbering air

In [156]:
old = np.unique(fluid_elements1)
new = list(range(1,len(np.unique(fluid_elements1))+1))
new_nodes=fluid_nodes[np.unique(fluid_elements1)-1,:]

In [157]:
dico1 = dict(zip(old,new))
new_elements=np.zeros((fluid_nelem1,4),dtype=int)
for e in range(fluid_nelem1):
    for i in range(4):
        new_elements[e,i]=dico1[fluid_elements1[e][i]]

In [158]:
fluid_elements1 = new_elements
fluid_nodes1    = new_nodes

In [159]:
new_elements=np.zeros((fluid_nelem5,4),dtype=int)
for e in range(fluid_nelem5):
    for i in range(4):
        new_elements[e,i]=dico1[fluid_elements5[e][i]]

In [160]:
fluid_elements5 = new_elements

# renumbering porous<br>
old = np.unique(fluid_elements2)<br>
new = list(range(1,len(np.unique(fluid_elements2))+1))<br>
new_nodes=fluid_nodes[scipy.unique(fluid_elements2)-1,:]<br>
<br>
dico2 = dict(zip(old,new))<br>
new_elements=scipy.zeros((fluid_nelem2,4),dtype=int)<br>
for e in range(fluid_nelem2):<br>
    for i in range(4):<br>
        new_elements[e,i]=dico2[fluid_elements2[e][i]]<br>
<br>
fluid_elements2 = new_elements<br>
fluid_nodes2    = new_nodes<br>
<br>
# renumbering porous external surfaces<br>
for i in range(len(IdNodesS4)):<br>
    IdNodesS4[i]=dico2[IdNodesS4[i]]

# Boundary conditions on air cavity<br>
IdNodesFixed_porous_us_x=IdNodesS4<br>
##IdNodesFixed_porous_us_y=scipy.unique(scipy.hstack([IdNodesS4,IdNodesS6]))<br>
IdNodesFixed_porous_us_y=IdNodesS4<br>
IdNodesFixed_porous_us_z=IdNodesS4<br>
IdNodesFixed_porous_uf_x=IdNodesS4<br>
IdNodesFixed_porous_uf_y=IdNodesS4<br>
IdNodesFixed_porous_uf_z=IdNodesS4<br>
<br>
Fixed_Dofs_porous = scipy.hstack([(IdNodesFixed_porous_us_x-1)*6,<br>
                                  (IdNodesFixed_porous_us_y-1)*6+1,<br>
                                  (IdNodesFixed_porous_us_z-1)*6+2,<br>
                                  (IdNodesFixed_porous_uf_x-1)*6+3,<br>
                                  (IdNodesFixed_porous_uf_y-1)*6+4,<br>
                                  (IdNodesFixed_porous_uf_z-1)*6+5<br>
                                  ])

get connectivity at air-porous interface

In [161]:
IdNodesS3_for_1=np.zeros(fluid_nnodes3,dtype=int)
#IdNodesS3_for_2=scipy.zeros(fluid_nnodes3,dtype=int)
for i in range(fluid_nnodes3):
    IdNodesS3_for_1[i]=dico1[IdNodesS3[i]] # for the air mesh
    #IdNodesS3_for_2[i]=dico2[IdNodesS3[i]] # for the porous mesh

In [162]:
InterfaceConnectivity=np.zeros((fluid_nelem3,3),dtype=int)
for e in range(fluid_nelem3):
    for i in range(3):
        InterfaceConnectivity[e,i]   = dico1[fluid_elements_S3[e,i]] # for the air mesh
        #InterfaceConnectivity[e,i+3] = dico2[fluid_elements_S3[e,i]] # for the porous mesh

In [163]:
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults(results_file+'_air_cavity_Mesh1',fluid_nodes1,fluid_elements1,4)
    #silex_lib_gmsh.WriteResults(results_file+'_porous_material_Mesh2',fluid_nodes2,fluid_elements2,4)
    silex_lib_gmsh.WriteResults(results_file+'_porous_air_interface_Mesh_surface3',fluid_nodes,fluid_elements_S3,2)
    #silex_lib_gmsh.WriteResults(results_file+'_porous_fixed_Mesh_surface4',fluid_nodes,fluid_elements_S4,2)
    silex_lib_gmsh.WriteResults(results_file+'_air_controlled_volume_Mesh5',fluid_nodes1,fluid_elements5,4)

    #silex_lib_gmsh.WriteResults(results_file+'_air_porous_interface1',fluid_nodes1,InterfaceConnectivity[:,range(3)],2)
    #silex_lib_gmsh.WriteResults(results_file+'_air_porous_interface2',fluid_nodes2,InterfaceConnectivity[:,range(3,6,1)],2)

############################################################<br>
Load structure mesh<br>
############################################################

In [164]:
struc_nodes        = silex_lib_gmsh.ReadGmshNodes(mesh_file+'_struc.msh',3)
struc_elements,tmp = silex_lib_gmsh.ReadGmshElements(mesh_file+'_struc.msh',2,6)
struc_boun,tmp     = silex_lib_gmsh.ReadGmshElements(mesh_file+'_struc.msh',1,7)

In [165]:
struc_nnodes   = struc_nodes.shape[0]
struc_nelem    = struc_elements.shape[0]
struc_ndof     = struc_nnodes*6

In [166]:
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults2(results_file+'_struc_mesh_surface_6',struc_nodes,struc_elements,2)
    silex_lib_gmsh.WriteResults2(results_file+'_struc_boun_mesh_line_7',struc_nodes,struc_boun,1)

In [167]:
if rank==0:
    print ("nnodes for structure=",struc_nnodes)
    print ("nelem for structure=",struc_nelem)

nnodes for structure= 3179
nelem for structure= 6143


Find the fixed dofs and the free dofs of the structure

In [168]:
tmp=scipy.sparse.find(struc_nodes[:,2]==0.0)# z=0
FixedStrucNodes=tmp[1]+1

In [169]:
FixedStrucDofUx=(FixedStrucNodes-1)*6
FixedStrucDofUy=(FixedStrucNodes-1)*6+1
FixedStrucDofUz=(FixedStrucNodes-1)*6+2
FixedStrucDofRx=(FixedStrucNodes-1)*6+3
FixedStrucDofRy=(FixedStrucNodes-1)*6+4
FixedStrucDofRz=(FixedStrucNodes-1)*6+5
#FixedStrucDofRx=[]
#FixedStrucDofRy=[]
#FixedStrucDofRz=[]

In [170]:
FixedStrucDof=np.hstack([FixedStrucDofUx,FixedStrucDofUy,FixedStrucDofUz,FixedStrucDofRx,FixedStrucDofRy,FixedStrucDofRz])

In [171]:
SolvedDofS=np.setdiff1d(range(struc_ndof),FixedStrucDof)

In [172]:
FS=np.zeros(struc_ndof)
#IddofLoadStructure=[(IdNodeLoadStructure-1)*6+2]
#FS[IddofLoadStructure]=1.0

############################################################<br>
Compute structure matrices<br>
############################################################

In [173]:
tic = time.perf_counter()

In [174]:
IIks,JJks,Vks,Vms=silex_lib_dkt.stiffnessmatrix(struc_nodes,struc_elements,material_Struc)

In [175]:
KSS = scipy.sparse.csc_matrix( (Vks,(IIks,JJks)), shape=(struc_ndof,struc_ndof) )
MSS = scipy.sparse.csc_matrix( (Vms,(IIks,JJks)), shape=(struc_ndof,struc_ndof) )

In [176]:
toc = time.perf_counter()
if rank==0:
    print ("time for computing structure:",toc-tic)

time for computing structure: 0.2544194559999937


################################################################<br>
Compute eigen modes of the structure<br>
################################################################

In [177]:
tic = time.perf_counter()

In [178]:
eigen_values_S,eigen_vectors_S= scipy.sparse.linalg.eigsh(KSS[SolvedDofS,:][:,SolvedDofS],nb_mode_S,MSS[SolvedDofS,:][:,SolvedDofS],sigma=0,which='LM')

In [179]:
eigen_values_S


array([5.26738172e+01, 2.40515149e+02, 1.43366957e+03, 2.32566915e+03,
       3.42599223e+03, 8.58230637e+03, 8.72401310e+03, 1.59962526e+04,
       1.81184432e+04, 2.50480165e+04, 2.68852014e+04, 3.41679672e+04,
       5.08418475e+04, 5.33865418e+04, 6.39319657e+04, 6.72486658e+04,
       8.16123322e+04, 8.77895328e+04, 1.07899317e+05, 1.09212557e+05,
       1.35104404e+05, 1.62203057e+05, 1.68665375e+05, 1.68755146e+05,
       1.89401111e+05, 2.04801193e+05, 2.08189387e+05, 2.53264469e+05,
       2.73529349e+05, 2.91879330e+05, 3.17209001e+05, 3.28041071e+05,
       3.58259564e+05, 3.86897951e+05, 3.94383381e+05, 4.34088898e+05,
       4.41626384e+05, 4.91527927e+05, 5.04823051e+05, 5.09581748e+05,
       5.67025740e+05, 6.17567019e+05, 6.43060198e+05, 6.77211846e+05,
       7.07106578e+05, 7.15498325e+05, 7.53809275e+05, 8.01976438e+05,
       8.24657340e+05, 8.66270474e+05])

In [180]:
freq_eigv_S=list(np.sqrt(eigen_values_S)/(2*np.pi))

In [181]:
toc = time.perf_counter()
if rank==0:
    print ("time for computing the structure modal basis:",toc-tic)

time for computing the structure modal basis: 4.1546703310000055


In [182]:
eigen_vector_S_list=[]
for i in range(nb_mode_S):
    Q=np.zeros(struc_ndof)
    Q[SolvedDofS]=eigen_vectors_S[:,i]
    disp=np.zeros((struc_nnodes,3))
    disp[range(struc_nnodes),0]=Q[list(range(0,struc_ndof,6))]
    disp[range(struc_nnodes),1]=Q[list(range(1,struc_ndof,6))]
    disp[range(struc_nnodes),2]=Q[list(range(2,struc_ndof,6))]
    eigen_vector_S_list.append(disp)

In [183]:
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults2(results_file+'_structure_modes',struc_nodes,struc_elements,2,[[eigen_vector_S_list,'nodal',3,'modes']])

################################################################<br>
compute level set<br>
################################################################

In [184]:
tic = time.perf_counter()

In [185]:
LevelSet,distance = silex_lib_xfem_acou_tet4.computelevelset(fluid_nodes1,struc_nodes,struc_elements)

In [186]:
toc = time.perf_counter()
if rank==0:
    print ("time to compute level set:",toc-tic)

time to compute level set: 0.36171761800000013


In [187]:
tic = time.perf_counter()

In [188]:
tangent_nodes,tangent_mesh=silex_lib_xfem_acou_tet4.buildtangentedgemesh(struc_nodes,struc_elements,struc_boun)

In [189]:
LevelSetTangent,tmp = silex_lib_xfem_acou_tet4.computelevelset(fluid_nodes1,tangent_nodes,tangent_mesh)

In [190]:
toc = time.perf_counter()
if rank==0:
    print ("time to compute tangent level set:",toc-tic)

time to compute tangent level set: 0.17525614299998438


In [191]:
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults2(results_file+'_LS_signed_distance',fluid_nodes1,fluid_elements1,4,[[[LevelSet],'nodal',1,'Level set']])
    silex_lib_gmsh.WriteResults2(results_file+'_LS_tangent_level_set',fluid_nodes1,fluid_elements1,4,[[[LevelSetTangent],'nodal',1,'Tangent level set']])
    silex_lib_gmsh.WriteResults2(results_file+'_LS_distance',fluid_nodes1,fluid_elements1,4,[[[distance],'nodal',1,'Distance']])
    silex_lib_gmsh.WriteResults2(results_file+'_LS_tangent_mesh',tangent_nodes,tangent_mesh,2)

################################################################<br>
Get enriched nodes and elements<br>
################################################################

In [192]:
tic = time.perf_counter()

In [193]:
LSEnrichedElements,NbLSEnrichedElements=silex_lib_xfem_acou_tet4.getenrichedelementsfromlevelset(fluid_elements1,LevelSet)
LSEnrichedElements=LSEnrichedElements[list(range(NbLSEnrichedElements))]
EnrichedElements,NbEnrichedElements=silex_lib_xfem_acou_tet4.getsurfenrichedelements(struc_nodes,struc_elements,fluid_nodes1,fluid_elements1[LSEnrichedElements])
EnrichedElements=np.unique(EnrichedElements[list(range(NbEnrichedElements))])
EnrichedElements=LSEnrichedElements[EnrichedElements-1]
toc = time.perf_counter()
if rank==0:
    print ("time to find surface enriched elements:",toc-tic)

time to find surface enriched elements: 2.861574994999984


In [194]:
tic = time.perf_counter()

In [195]:
EdgeEnrichedElements,nbenrelts = silex_lib_xfem_acou_tet4.getedgeenrichedelements(struc_nodes,struc_boun,fluid_nodes1,fluid_elements1)
EdgeEnrichedElements=np.unique(EdgeEnrichedElements[list(range(nbenrelts))])-1

In [196]:
EdgeEnrichedElementsInAllMesh,nbEdgeEnrichedElementsInAllMesh=silex_lib_xfem_acou_tet4.getenrichedelementsfromlevelset(fluid_elements1,LevelSetTangent)
EdgeEnrichedElementsInAllMesh=np.unique(EdgeEnrichedElementsInAllMesh[list(range(nbEdgeEnrichedElementsInAllMesh))])

In [197]:
toc = time.perf_counter()
if rank==0:
    print ("time to find edge enriched elements:",toc-tic)

time to find edge enriched elements: 0.48864829699999746


In [198]:
HeavisideEnrichedElements=np.setdiff1d(EnrichedElements,EdgeEnrichedElements)

In [199]:
AllElementsExceptEdgeEnrichedElements=np.setdiff1d(range(fluid_nelem1),EdgeEnrichedElements)

In [200]:
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults2(results_file+'_LSenriched_elements',fluid_nodes1,fluid_elements1[LSEnrichedElements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_enriched_elements',fluid_nodes1,fluid_elements1[EnrichedElements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_edge_enriched_elements',fluid_nodes1,fluid_elements1[EdgeEnrichedElements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_edge_enriched_elements_in_all_mesh',fluid_nodes1,fluid_elements1[EdgeEnrichedElementsInAllMesh],4)
    silex_lib_gmsh.WriteResults2(results_file+'_heaviside_enriched_elements',fluid_nodes1,fluid_elements1[HeavisideEnrichedElements],4)

################################################################<br>
Compute coupling STRUCTURE / AIR terms<br>
################################################################

In [201]:
tic = time.perf_counter()

In [202]:
IIc1,JJc1,Vc1=silex_lib_xfem_acou_tet4.computexfemcoupling1(fluid_nodes1,struc_nodes,fluid_elements1,struc_elements,EnrichedElements)
IIc2,JJc2,Vc2=silex_lib_xfem_acou_tet4.computexfemcoupling2(fluid_nodes1,struc_nodes,fluid_elements1,struc_elements,EnrichedElements,LevelSet)

In [203]:
CSA=0.5*scipy.sparse.csc_matrix( (Vc1,(IIc1,JJc1)), shape=(struc_ndof,fluid_ndof1) )+0.5*scipy.sparse.csc_matrix( (Vc2,(IIc2,JJc2)), shape=(struc_ndof,fluid_ndof1) )

In [204]:
toc = time.perf_counter()
if rank==0:
    print ("time to compute coupling matrices:",toc-tic)

time to compute coupling matrices: 1.9047934949999785


############################################################<br>
Compute Standard Fluid Matrices<br>
############################################################

In [205]:
tic = time.perf_counter()

In [206]:
IIf,JJf,Vffk,Vffm=silex_lib_xfem_acou_tet4.globalacousticmatrices(fluid_elements1,fluid_nodes1,celerity,rho)

In [207]:
KFF=scipy.sparse.csc_matrix( (Vffk,(IIf,JJf)), shape=(fluid_ndof1,fluid_ndof1) )
MFF=scipy.sparse.csc_matrix( (Vffm,(IIf,JJf)), shape=(fluid_ndof1,fluid_ndof1) )

In [208]:
SolvedDofF=list(range(fluid_ndof1))

In [209]:
SolvedDofB=np.hstack([IdNodesS3_for_1-1,9-1]) # 9 : node number where acoustic source is imposed
SolvedDofI=np.setdiff1d(SolvedDofF,SolvedDofB)

############################################################<br>
Compute impedance matrices<br>
############################################################<br>
rint(silex_lib_porous_tet4_fortran.stiffnessimpedanceparoi.__doc__)

In [210]:
IIimp,JJimp,Vimp=silex_lib_porous_tet4_fortran.stiffnessimpedanceparoi(fluid_nodes1,InterfaceConnectivity)

In [211]:
CII=scipy.sparse.csc_matrix( (Vimp,(IIimp,JJimp)), shape=(fluid_ndof1,fluid_ndof1) )

################################################################<br>
Compute Heaviside enrichment<br>
################################################################

In [212]:
tic = time.perf_counter()

nrichednodes = scipy.unique(fluid_elements1[HeavisideEnrichedElements])

In [213]:
Enrichednodes = np.unique(fluid_elements1[EnrichedElements])

In [214]:
NegativeLSelements,PositiveLSelements,NegativeLStgtElements,PositiveLStgtElements,nbNegLS,nbPosLS,nbNegLSt,nbPosLSt=silex_lib_xfem_acou_tet4.getpositivenegativeelts(fluid_elements1,LevelSet,LevelSetTangent)

In [215]:
NegativeLSelements=NegativeLSelements[list(range(nbNegLS))]
PositiveLSelements=PositiveLSelements[list(range(nbPosLS))]
NegativeLStgtElements=NegativeLStgtElements[list(range(nbNegLSt))]
PositiveLStgtElements=PositiveLStgtElements[list(range(nbPosLSt))]

Iaa,JJaa,IIaf,JJaf,Vaak,Vaam,Vafk,Vafm=silex_lib_xfem_acou_tet4.globalxfemacousticmatrices(fluid_elements1[AllElementsExceptEdge],fluid_nodes1,LevelSet,celerity,rho)

In [216]:
IIaa,JJaa,IIaf,JJaf,Vaak,Vaam,Vafk,Vafm=silex_lib_xfem_acou_tet4.globalxfemacousticmatrices(fluid_elements1[NegativeLStgtElements],fluid_nodes1,LevelSet,celerity,rho)
#IIaa,JJaa,IIaf,JJaf,Vaak,Vaam,Vafk,Vafm=silex_lib_xfem_acou_tet4.globalxfemacousticmatrices(fluid_elements1,fluid_nodes1,LevelSet,celerity,rho)

In [217]:
KAAheaviside = scipy.sparse.csc_matrix( (Vaak,(IIaa,JJaa)), shape=(fluid_ndof1,fluid_ndof1) )
MAAheaviside = scipy.sparse.csc_matrix( (Vaam,(IIaa,JJaa)), shape=(fluid_ndof1,fluid_ndof1) )
KAFheaviside = scipy.sparse.csc_matrix( (Vafk,(IIaf,JJaf)), shape=(fluid_ndof1,fluid_ndof1) )
MAFheaviside = scipy.sparse.csc_matrix( (Vafm,(IIaf,JJaf)), shape=(fluid_ndof1,fluid_ndof1) )

In [218]:
SolvedDofA=Enrichednodes-1

In [219]:
toc = time.perf_counter()
if rank==0:
    print ("time to compute Heaviside enrichment:",toc-tic)

time to compute Heaviside enrichment: 0.7502792239999962


################################################################<br>
Compute Edge enrichment<br>
################################################################

In [220]:
tic = time.perf_counter()

In [221]:
PartiallyPositiveLStgtElements=np.hstack([PositiveLStgtElements,EdgeEnrichedElementsInAllMesh])

I,JJ,vkaa,vmaa,vkfa,vmfa = silex_lib_xfem_acou_tet4.computeedgeenrichment(fluid_nodes1,fluid_elements1[EdgeEnrichedElements],LevelSet,LevelSetTangent,celerity,rho)

In [222]:
II,JJ,vkaa,vmaa,vkfa,vmfa = silex_lib_xfem_acou_tet4.computeedgeenrichment(fluid_nodes1,fluid_elements1[PartiallyPositiveLStgtElements],LevelSet,LevelSetTangent,celerity,rho)
#II,JJ,vkaa,vmaa,vkfa,vmfa = silex_lib_xfem_acou_tet4.computeedgeenrichment(fluid_nodes1,fluid_elements1,LevelSet,LevelSetTangent,celerity,rho)

In [223]:
KAAedge = scipy.sparse.csc_matrix( (vkaa,(II,JJ)), shape=(fluid_ndof1,fluid_ndof1) )
MAAedge = scipy.sparse.csc_matrix( (vmaa,(II,JJ)), shape=(fluid_ndof1,fluid_ndof1) )
KAFedge = scipy.sparse.csc_matrix( (vkfa,(II,JJ)), shape=(fluid_ndof1,fluid_ndof1) )
MAFedge = scipy.sparse.csc_matrix( (vmfa,(II,JJ)), shape=(fluid_ndof1,fluid_ndof1) )

In [224]:
toc = time.perf_counter()
if rank==0:
    print ("time to compute edge enrichment:",toc-tic)

time to compute edge enrichment: 0.6287696830000016


In [225]:
KAA=KAAheaviside+KAAedge
MAA=MAAheaviside+MAAedge
KAF=KAFheaviside+KAFedge
MAF=MAFheaviside+MAFedge

In [226]:
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults2(results_file+'_NegativeLSelements',fluid_nodes1,fluid_elements1[NegativeLSelements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_PositiveLSelements',fluid_nodes1,fluid_elements1[PositiveLSelements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_NegativeLStgtElements',fluid_nodes1,fluid_elements1[NegativeLStgtElements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_PositiveLStgtElements',fluid_nodes1,fluid_elements1[PositiveLStgtElements],4)
    silex_lib_gmsh.WriteResults2(results_file+'_PartiallyPositiveLStgtElements',fluid_nodes1,fluid_elements1[PartiallyPositiveLStgtElements],4)

################################################################<br>
Compute eigen modes of the fluid: internal dof I<br>
################################################################

In [227]:
tic = time.perf_counter()

In [228]:
if rank==0:
    print ("Compute fluid modes")
eigen_values_I,eigen_vectors_I= scipy.sparse.linalg.eigsh(KFF[SolvedDofI,:][:,SolvedDofI],nb_mode_F,MFF[SolvedDofI,:][:,SolvedDofI],sigma=0,which='LM')

Compute fluid modes


In [229]:
freq_eigv_I=list(np.sqrt(eigen_values_I)/(2*np.pi))
eigen_vector_F_list=[]
for i in range(nb_mode_F):
    tmp=np.zeros((fluid_ndof1) , dtype='float')
    tmp[SolvedDofI]=eigen_vectors_I[:,i].real
    eigen_vector_F_list.append(tmp)

In [230]:
if rank==0:
    print ("LAST fluid eigen frequencies : ",freq_eigv_I[-1])
 
if (flag_write_gmsh_results==1) and (rank==0):
    silex_lib_gmsh.WriteResults2(results_file+'_fluid_modes',fluid_nodes1,fluid_elements1,4,[[eigen_vector_F_list,'nodal',1,'pressure']])

LAST fluid eigen frequencies :  271.9665653033819


In [231]:
toc = time.perf_counter()
if rank==0:
    print ("time for computing the fluid modes:",toc-tic)

time for computing the fluid modes: 23.905065385


################################################################<br>
Compute Psi_IA for the fluid: Psi_IA = - KII^{-1} * KIA<br>
################################################################

In [232]:
tic = time.perf_counter()

In [233]:
if rank==0:
    print ("Compute PSI_IA")

Compute PSI_IA


In [234]:
omega_cst=1.0*2.0*np.pi
MySolve = scipy.sparse.linalg.factorized( KFF[SolvedDofI,:][:,SolvedDofI]-(omega_cst**2)*MFF[SolvedDofI,:][:,SolvedDofI] ) # Makes LU decomposition.

In [235]:
if rank==0:
    print("LU decomposition has been made")

LU decomposition has been made


In [236]:
if rank==0:
    Psi_IA=np.zeros((len(SolvedDofI),len(SolvedDofA)))

In [237]:
i=0
j=0
while j<len(SolvedDofA):
    j=i+rank
    if j<len(SolvedDofA):
        One_dof=[SolvedDofA[j]]
        KIA_i_column=-KAF[SolvedDofI,:][:,One_dof]
        #tmp=scipy.zeros(len(SolvedDofF))
        #tmp[SolvedDofI]=KIA_i_column.todense()
        Xi=MySolve( KIA_i_column.todense() )
        if rank!=0:
            comm.send([Xi,j], dest=0, tag=11)
        if rank==0:
            for k in range(nproc):
                if k!=0:
                    if i+k<len(SolvedDofA):
                        [Xi,j]=comm.recv(source=k, tag=11)
                        Psi_IA[:,j]=np.array(Xi)[:,0]
                else:
                    Psi_IA[:,j]=np.array(Xi)[:,0]
        i=i+nproc

In [238]:
if rank==0:
    print("End of PSI_IA computing, send to other proc.")

End of PSI_IA computing, send to other proc.


In [239]:
if rank==0:
    for i in range(nproc):
        if i!=0:
            comm.send(Psi_IA, dest=i, tag=11)
if rank!=0:
    Psi_IA=comm.recv(source=0, tag=11)

In [240]:
Psi_IA=scipy.sparse.csc_matrix(Psi_IA)
toc = time.perf_counter()

In [241]:
if rank==0:
    print ("time to compute PSI_IA:",toc-tic)

time to compute PSI_IA: 11.584351440999995


if rank==0:<br>
    eigen_vector_I_list=[]<br>
    for i in range(len(SolvedDofA)):<br>
        tmp=scipy.zeros((fluid_ndof1) , dtype='float')<br>
        tmp[SolvedDofI]=Psi_IA[:,i].todense()<br>
        eigen_vector_I_list.append(tmp)<br>
<br>
if (flag_write_gmsh_results==1) and (rank==0):<br>
    silex_lib_gmsh.WriteResults2(results_file+'_Psi_IA',fluid_nodes1,fluid_elements1,4,[[eigen_vector_I_list,'nodal',1,'PSI IA pressure']])

################################################################<br>
Compute Psi_IB for the fluid: Psi_IB = - KII^{-1} * KIB<br>
################################################################

In [242]:
tic = time.perf_counter()

In [243]:
if rank==0:
    print ("Compute PSI_IB")

Compute PSI_IB


In [244]:
if rank==0:
    Psi_IB=np.zeros((len(SolvedDofI),len(SolvedDofB)))

In [245]:
i=0
j=0
while j<len(SolvedDofB):
    j=i+rank
    if j<len(SolvedDofB):
        One_dof=[SolvedDofB[j]]
        KIB_i_column=-KFF[SolvedDofI,:][:,One_dof]
        Xi=MySolve( KIB_i_column.todense() )
        if rank!=0:
            comm.send([Xi,j], dest=0, tag=11)
        if rank==0:
            for k in range(nproc):
                if k!=0:
                    if i+k<len(SolvedDofB):
                        [Xi,j]=comm.recv(source=k, tag=11)
                        Psi_IB[:,j]=np.array(Xi)[:,0]
                else:
                    Psi_IB[:,j]=np.array(Xi)[:,0]
        i=i+nproc

In [246]:
if rank==0:
    print("End of PSI_IB computing, send to other proc.")

End of PSI_IB computing, send to other proc.


In [247]:
if rank==0:
    for i in range(nproc):
        if i!=0:
            comm.send(Psi_IB, dest=i, tag=11)
if rank!=0:
    Psi_IB=comm.recv(source=0, tag=11)

In [248]:
Psi_IB=scipy.sparse.csc_matrix(Psi_IB)
toc = time.perf_counter()

In [249]:
if rank==0:
    print ("time to compute PSI_FB:",toc-tic)

time to compute PSI_FB: 10.191547730000025


if rank==0:<br>
    eigen_vector_I_list=[]<br>
    for i in range(len(SolvedDofB)):<br>
        tmp=scipy.zeros((fluid_ndof1) , dtype='float')<br>
        tmp[SolvedDofI]=Psi_IB[:,i].todense()<br>
        eigen_vector_I_list.append(tmp)<br>
<br>
if (flag_write_gmsh_results==1) and (rank==0):<br>
    silex_lib_gmsh.WriteResults2(results_file+'_Psi_IB',fluid_nodes1,fluid_elements1,4,[[eigen_vector_I_list,'nodal',1,'PSI IB pressure']])

################################################################<br>
Construct structure projection<br>
################################################################

In [250]:
VK_diag_nn = eigen_values_S
VM_diag_nn = eigen_values_S/eigen_values_S
IIDnn = list(range(nb_mode_S))
JJDnn = list(range(nb_mode_S))

In [251]:
Knn= scipy.sparse.csc_matrix( (VK_diag_nn,(IIDnn,JJDnn)), shape=(nb_mode_S,nb_mode_S) )
Mnn= scipy.sparse.csc_matrix( (VM_diag_nn,(IIDnn,JJDnn)), shape=(nb_mode_S,nb_mode_S) )

Projection of air-structure coupling on structure modal basis

In [252]:
CnA = eigen_vectors_S.T*CSA[SolvedDofS,:][:,SolvedDofA]

################################################################<br>
Compute structure damping matrix<br>
################################################################

In [253]:
VDnn = 2.0*modal_damping_S*np.sqrt(eigen_values_S)
IIDnn = list(range(nb_mode_F+len(SolvedDofB)+len(SolvedDofA),nb_mode_F+len(SolvedDofB)+len(SolvedDofA)+nb_mode_S))
JJDnn = list(range(nb_mode_F+len(SolvedDofB)+len(SolvedDofA),nb_mode_F+len(SolvedDofB)+len(SolvedDofA)+nb_mode_S))

In [254]:
totaldofs=nb_mode_F+len(SolvedDofB)+len(SolvedDofA)+nb_mode_S

In [255]:
D = scipy.sparse.coo_matrix( (VDnn,(IIDnn,JJDnn)), shape=(totaldofs,totaldofs) )

################################################################<br>
Construct the whole system<br>
################################################################

Fluid part

In [256]:
tic = time.perf_counter()

In [257]:
VK_diag_mm = eigen_values_I
VM_diag_mm = eigen_values_I/eigen_values_I
IIDmm = list(range(nb_mode_F))
JJDmm = list(range(nb_mode_F))

hiFm=eigen_vectors_F

In [258]:
K_diag_mm= scipy.sparse.csc_matrix( (VK_diag_mm,(IIDmm,JJDmm)), shape=(nb_mode_F,nb_mode_F) )
M_diag_mm= scipy.sparse.csc_matrix( (VM_diag_mm,(IIDmm,JJDmm)), shape=(nb_mode_F,nb_mode_F) )
Khat_AA = KAA[SolvedDofA,:][:,SolvedDofA]+Psi_IA.T*KAF[SolvedDofI,:][:,SolvedDofA]
Khat_BB = KFF[SolvedDofB,:][:,SolvedDofB]+Psi_IB.T*KFF[SolvedDofI,:][:,SolvedDofB]

In [259]:
Khat_BA = KFF[SolvedDofB,:][:,SolvedDofI]*Psi_IA

In [260]:
toc = time.perf_counter()
if rank==0:
    print ("time to compute Khat_hat:",toc-tic)

time to compute Khat_hat: 0.16135973600000852


In [261]:
tic = time.perf_counter()

In [262]:
Mstar_IA = MFF[SolvedDofI,:][:,SolvedDofI]*Psi_IA+MAF[SolvedDofI,:][:,SolvedDofA]
Mstar_IB = MFF[SolvedDofI,:][:,SolvedDofI]*Psi_IB+MFF[SolvedDofI,:][:,SolvedDofB]

In [263]:
toc = time.perf_counter()
if rank==0:
    print ("time to compute Mstar:",toc-tic)

time to compute Mstar: 1.0849516270000095


In [264]:
tic = time.perf_counter()

In [265]:
Mhat_AA = MAA[SolvedDofA,:][:,SolvedDofA]+scipy.sparse.csc_matrix((Psi_IA.T).todense()*Mstar_IA.todense())+MAF[SolvedDofA,:][:,SolvedDofI]*Psi_IA
Mhat_BB = MFF[SolvedDofB,:][:,SolvedDofB]+scipy.sparse.csc_matrix((Psi_IB.T).todense()*Mstar_IB.todense())+MFF[SolvedDofB,:][:,SolvedDofI]*Psi_IB

hat_AA = MAA[SolvedDofA,:][:,SolvedDofA]+Psi_IA.T*Mstar_IA+MAF[SolvedDofA,:][:,SolvedDofI]*Psi_IA<br>
hat_BB = MFF[SolvedDofB,:][:,SolvedDofB]+Psi_IB.T*Mstar_IB+MFF[SolvedDofB,:][:,SolvedDofI]*Psi_IB

hat_AA = MAA[SolvedDofA,:][:,SolvedDofA]+scipy.dot((Psi_IA.T).todense(),Mstar_IA.todense())+MAF[SolvedDofA,:][:,SolvedDofI]*Psi_IA<br>
hat_BB = MFF[SolvedDofB,:][:,SolvedDofB]+scipy.dot((Psi_IB.T).todense(),Mstar_IB.todense())+MFF[SolvedDofB,:][:,SolvedDofI]*Psi_IB

In [266]:
toc = time.perf_counter()
if rank==0:
    print ("time to compute Mhat:",toc-tic)

time to compute Mhat: 0.2573331509999832


mP=PhiFm.T*CPF[SolvedDofP,:][:,SolvedDofF].T<br>
AP=Psi_FA.T*CPF[SolvedDofP,:][:,SolvedDofF].T

In [267]:
eigen_vectors_I=scipy.sparse.csc_matrix(eigen_vectors_I)

In [268]:
Mhat_mA = eigen_vectors_I.T*Mstar_IA
Mhat_mB = eigen_vectors_I.T*Mstar_IB

In [269]:
Mhat_BA = scipy.sparse.csc_matrix((Psi_IB.T).todense()*Mstar_IA.todense())+MFF[SolvedDofB,:][:,SolvedDofI]*Psi_IA

################################################################<br>
Build Second member<br>
################################################################

To impose the load on the fluid:<br>
fluid node number 1

In [270]:
UF = np.zeros(fluid_ndof1,dtype=float)
UF[9-1]=3.1250E-05

olvedDof = scipy.hstack([SolvedDofF,SolvedDofA+fluid_ndof1,SolvedDofP+2*fluid_ndof1])

In [271]:
Freduced_F=np.hstack([np.zeros(nb_mode_F,dtype=float),UF[SolvedDofB]])

############################################################<br>
FRF computation<br>
############################################################

In [272]:
Flag_frf_analysis=1
frequencies=[]
frf=[]

In [273]:
print(nb_freq_step)
print(nb_freq_step_per_proc)

800
800


In [274]:
if (Flag_frf_analysis == 1):
    print("Proc. ", rank, " / time at the beginning of the FRF:", time.ctime())
    if rank == 0:
        print('nb of total dofs: ', nb_mode_F +
              len(SolvedDofB)+len(SolvedDofA)+nb_mode_S,)
    press_save = []
    disp_save = []
    frfsave = []
    for i in range(nb_freq_step_per_proc):
        freq = freq_ini+i*nproc*deltafreq+rank*deltafreq
        frequencies.append(freq)
        omega = 2*np.pi*freq
        print("proc number {} frequency={} (it. {})".format(rank, freq, i))
        tic = time.perf_counter()
        Kimp = -(omega**2/(k_imp_paroi-1j*omega*d_imp_paroi))*CII
        K = scipy.sparse.construct.bmat([[K_diag_mm, None,       None,                           None],
                                         [None,     Khat_BB+Kimp[SolvedDofB, :][:, SolvedDofB],
                                          Khat_BA,                           None],
                                         [None,     Khat_BA.T,  Khat_AA,
                                          None],
                                         [None,     None,       -CnA, Knn]
                                         ]
                                        )

        M = scipy.sparse.construct.bmat([[M_diag_mm,    Mhat_mB,    Mhat_mA,                          None],
                                         [Mhat_mB.T,    Mhat_BB,    Mhat_BA,
                                          None],
                                         [Mhat_mA.T,    Mhat_BA.T,  Mhat_AA,
                                          CnA.T],
                                         [None,         None,       None,
                                          Mnn]
                                         ]
                                        )

        F = np.append(omega**2*Freduced_F,
                      np.zeros((len(SolvedDofA)+nb_mode_S), dtype='c16'))
        sol = mumps.spsolve(scipy.sparse.coo_matrix(
            K-(omega**2)*M+omega*D*1j, dtype='c16'), F, comm=mycomm)
        alpha_m = sol[list(range(nb_mode_F))]
        P_B = sol[list(range(nb_mode_F, nb_mode_F+len(SolvedDofB), 1))]
        P_A = sol[list(range(nb_mode_F+len(SolvedDofB),
                       nb_mode_F+len(SolvedDofB)+len(SolvedDofA), 1))]

        P_I = eigen_vectors_I*alpha_m+Psi_IB*P_B+Psi_IA*P_A

        press = np.zeros(fluid_ndof1, dtype=complex)
        press[SolvedDofI] = P_I
        press[SolvedDofB] = P_B
        enrichment = np.zeros(fluid_ndof1, dtype=complex)
        enrichment[SolvedDofA] = P_A
        CorrectedPressure = np.array(press)
        CorrectedPressure[SolvedDofA] = CorrectedPressure[SolvedDofA].T + \
            np.array(enrichment[SolvedDofA]*np.sign(LevelSet[SolvedDofA]).T)
        frf.append(silex_lib_xfem_acou_tet4.computecomplexquadratiquepressure(
            fluid_elements5, fluid_nodes1, CorrectedPressure))
        # frf.append(silex_lib_xfem_acou_tet4.computexfemcomplexquadratiquepressure(fluid_elements5,fluid_nodes1,press,enrichment,LevelSet,LevelSetTangent))
        if (flag_write_gmsh_results == 1) and (rank == 0):
            press_save.append(CorrectedPressure.real)
            alpha_n = sol[list(range(nb_mode_F+len(SolvedDofB)+len(SolvedDofA),
                               nb_mode_F+len(SolvedDofB)+len(SolvedDofA)+nb_mode_S, 1))].real
            Q = np.zeros((struc_ndof), dtype=float)
            Q[SolvedDofS] = eigen_vectors_S@alpha_n
            disp = np.zeros((struc_nnodes, 3))
            disp[range(struc_nnodes), 0] = Q[list(range(0, struc_ndof, 6))]
            disp[range(struc_nnodes), 1] = Q[list(range(1, struc_ndof, 6))]
            disp[range(struc_nnodes), 2] = Q[list(range(2, struc_ndof, 6))]
            disp_save.append(disp)

    frfsave = [frequencies, frf]
    if rank != 0:
        comm.send(frfsave, dest=0, tag=11)
    print("Proc. ", rank, " / time at the end of the FRF:", time.ctime())
    if (flag_write_gmsh_results == 1) and (rank == 0):
        silex_lib_gmsh.WriteResults2(results_file+str(rank)+'_results_fluid_frf',
                                     fluid_nodes1, fluid_elements1, 4, [[press_save, 'nodal', 1, 'pressure']])
        silex_lib_gmsh.WriteResults2(results_file+str(rank)+'_results_struct_frf',
                                     struc_nodes, struc_elements, 2, [[disp_save, 'nodal', 3, 'displacement']])

    # Save the FRF problem
    Allfrequencies = np.zeros(nb_freq_step)
    Allfrf = np.zeros(nb_freq_step)
    k = 0
    if rank == 0:
        for i in range(nproc):
            if i == 0:
                data = frfsave
            else:
                data = comm.recv(source=i, tag=11)
            for j in range(len(data[0])):
                Allfrequencies[k] = data[0][j]
                Allfrf[k] = data[1][j]
                k = k+1
        Allfrequencies, Allfrf = zip(*sorted(zip(Allfrequencies, Allfrf)))
        Allfrfsave = [np.array(list(Allfrequencies)), np.array(list(Allfrf))]
        f = open(results_file+'_results.frf', 'wb')
        pickle.dump(Allfrfsave, f)
        f.close()
        print('Last eigenfrequency in fluid basis: ', freq_eigv_I[-1])
        print('Last eigenfrequency in structure basis: ', freq_eigv_S[-1])
        print('nb of total dofs: ', nb_mode_F +
              len(SolvedDofB)+len(SolvedDofA)+nb_mode_S,)


Proc.  0  / time at the beginning of the FRF: Fri Feb 11 14:59:40 2022
nb of total dofs:  1021
proc number 0 frequency=10.0 (it. 0)
proc number 0 frequency=10.237797246558198 (it. 1)
proc number 0 frequency=10.475594493116395 (it. 2)
proc number 0 frequency=10.713391739674593 (it. 3)
proc number 0 frequency=10.951188986232792 (it. 4)
proc number 0 frequency=11.188986232790988 (it. 5)
proc number 0 frequency=11.426783479349186 (it. 6)
proc number 0 frequency=11.664580725907385 (it. 7)
proc number 0 frequency=11.902377972465581 (it. 8)
proc number 0 frequency=12.14017521902378 (it. 9)
proc number 0 frequency=12.377972465581978 (it. 10)
proc number 0 frequency=12.615769712140175 (it. 11)
proc number 0 frequency=12.853566958698373 (it. 12)
proc number 0 frequency=13.091364205256571 (it. 13)
proc number 0 frequency=13.32916145181477 (it. 14)
proc number 0 frequency=13.566958698372966 (it. 15)
proc number 0 frequency=13.804755944931165 (it. 16)
proc number 0 frequency=14.042553191489361 (it.